In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
from pyspark.sql import functions as F
from manga_recsys.spark import get_spark

spark = get_spark()
df = spark.read.json("../data/raw/2022-12-17-mangadex-group.ndjson")
df.printSchema()
df.show(n=3, truncate=60, vertical=True)

root
 |-- attributes: struct (nullable = true)
 |    |-- altNames: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- en: string (nullable = true)
 |    |-- contactEmail: string (nullable = true)
 |    |-- createdAt: string (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- discord: string (nullable = true)
 |    |-- focusedLanguages: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- inactive: boolean (nullable = true)
 |    |-- ircChannel: string (nullable = true)
 |    |-- ircServer: string (nullable = true)
 |    |-- locked: boolean (nullable = true)
 |    |-- mangaUpdates: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- official: boolean (nullable = true)
 |    |-- publishDelay: string (nullable = true)
 |    |-- twitter: string (nullable = true)
 |    |-- updatedAt: string (nullable = true)
 |    |-- verified: boolean (nullable = true)
 |    |-- v

In [8]:
alt_names = (
    df.select("id", F.explode("attributes.altNames"))
    .withColumn("altNames", F.col("col.en"))
    .drop("col")
)
alt_names.show(truncate=False)
alt_names.groupBy("id").count().where(F.col("count") > 1).count()

+------------------------------------+--------------------+
|id                                  |altNames            |
+------------------------------------+--------------------+
|80d82a0e-4ed3-41b5-8f2f-9eb2a06e8daf|Sol Cafe Scans      |
|fdad9851-164f-43fc-9ab0-25b8054c99ed|FLAME-SCANS         |
|26e19083-dda2-4a4e-b706-51546e3cb374|Le Cercle du Webtoon|
|56ddcbfa-ac51-43ee-8055-5f93d3757386|Manga-TR            |
|56ddcbfa-ac51-43ee-8055-5f93d3757386|MTR                 |
|43a13d2b-21d5-4453-ad78-cba547b7e879|mangacultivator     |
|134873ad-5513-4fbe-83bb-402a1d18ed7e|MissMystery94       |
|134873ad-5513-4fbe-83bb-402a1d18ed7e|Miss Mystery        |
|d7eb99fd-4444-4fbc-bd3c-c55ed35188e2|Ow! Scan            |
|ba83b2a6-7b2f-431b-9cae-4436cd8cce42|AnimaRegia Scantrad |
|70607404-c6c9-4467-b372-b174bb6097ad|Студия Гротеск      |
|8ca19ad0-f0ce-46bd-a3a9-46932cfa3d83|Naabii              |
|efd8e5ca-eef9-43c6-9a1a-d5199c43d758|FwPA                |
|b2def6de-2728-44f8-8c0f-a307584159f7|Żó

58

In [10]:
relationships = (
    df.select("id", F.explode("relationships"))
    .select("id", F.col("col.id").alias("group_id"), F.col("col.type"))
    .drop("col")
)
relationships.show(truncate=False)

+------------------------------------+------------------------------------+------+
|id                                  |group_id                            |type  |
+------------------------------------+------------------------------------+------+
|c6931ee7-b4cd-44da-a52b-c8d1a90db4d2|7c68e07d-d2f4-4021-82dc-054d42241aeb|leader|
|c6931ee7-b4cd-44da-a52b-c8d1a90db4d2|7c68e07d-d2f4-4021-82dc-054d42241aeb|member|
|145f9110-0a6c-4b71-8737-6acb1a3c5da4|d2ae45e0-b5e2-4e7f-a688-17925c2d7d6b|leader|
|145f9110-0a6c-4b71-8737-6acb1a3c5da4|d2ae45e0-b5e2-4e7f-a688-17925c2d7d6b|member|
|7f4ea5d0-6af4-48a4-b56c-c7240668096b|f4ef0210-c7b8-48fb-867a-f63dff560eeb|leader|
|7f4ea5d0-6af4-48a4-b56c-c7240668096b|f4ef0210-c7b8-48fb-867a-f63dff560eeb|member|
|1d1bf070-9e6d-490c-b20b-e69c06964338|5a64e325-ae24-4449-bd9e-8b726b6b507f|leader|
|1d1bf070-9e6d-490c-b20b-e69c06964338|192bc266-def4-46de-8d2d-bc37b60eaa68|member|
|1d1bf070-9e6d-490c-b20b-e69c06964338|286be38c-cc89-4fb8-9883-959498140fcf|member|
|1d1